In [1]:
! pip install openai==0.28

In [11]:
import os
import openai       

openai.api_key = "sk-proj-EZ2HXokcnX8O6Zh6OgQCT3BlbkFJSmT5ONnLxQrPGd6HYA9Q"
os.environ["OPENAI_API_KEY"] = "sk-proj-EZ2HXokcnX8O6Zh6OgQCT3BlbkFJSmT5ONnLxQrPGd6HYA9Q"

In [2]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500)

In [3]:
from langchain.vectorstores.faiss import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [4]:
textpath = CSVLoader('sales_data.csv',encoding="utf-8").load()
documents = text_splitter.split_documents(textpath)
db = FAISS.from_documents(documents, HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2"))

c:\Users\GM\Anaconda3\envs\bedrock\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
user_message  = "tell me the total amount for beauty category"
docs = db.similarity_search(user_message, k= 10)

In [6]:
knowledge_base = []
for documents in docs:
  knowledge_base.append(
      {
          "Information" : documents.page_content
      }
)

In [7]:
print(knowledge_base)

[{'Information': '\ufeffTransaction ID: 441\nDate: 10-10-2023\nCustomer ID: CUST441\nGender: Male\nAge: 57\nProduct Category: Beauty\nQuantity: 4\nPrice per Unit: 300\nTotal Amount: 1200'}, {'Information': '\ufeffTransaction ID: 660\nDate: 29-04-2023\nCustomer ID: CUST660\nGender: Female\nAge: 38\nProduct Category: Beauty\nQuantity: 2\nPrice per Unit: 500\nTotal Amount: 1000'}, {'Information': '\ufeffTransaction ID: 589\nDate: 12-04-2023\nCustomer ID: CUST589\nGender: Female\nAge: 36\nProduct Category: Beauty\nQuantity: 2\nPrice per Unit: 500\nTotal Amount: 1000'}, {'Information': '\ufeffTransaction ID: 554\nDate: 12-11-2023\nCustomer ID: CUST554\nGender: Female\nAge: 46\nProduct Category: Beauty\nQuantity: 3\nPrice per Unit: 50\nTotal Amount: 150'}, {'Information': '\ufeffTransaction ID: 276\nDate: 02-10-2023\nCustomer ID: CUST276\nGender: Female\nAge: 21\nProduct Category: Beauty\nQuantity: 4\nPrice per Unit: 25\nTotal Amount: 100'}, {'Information': '\ufeffTransaction ID: 97\nDate: 1

In [8]:
message = [
    {
        "role":"system",
        "content":f"""You are a smart chatbot You need to refer the knowledge base and respond to the user query,
        You need to be truthful and accurate with your response make sure your responses are derived only by refering the knowledge base,
        In situation where you dont have the answer to the user query present in the knowledge base, say 'sorry I dont have information.'
        You are tasked to answer user queries sales dataset and the information is stored in the below knowledge base,
        So refer to that and give answer to the user query, Return only the answer to the user query as your final response\n\nKnowledge Base: {knowledge_base}""",
    },
    {
        "role":"user",
        "content": user_message
    }
]

In [12]:
completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=message,
            temperature=0.0,
            max_tokens=500,
        )
output = completion["choices"][0]["message"]
response = output["content"]

In [13]:
print(response)

Total amount for Beauty category is 7600
